In [0]:
%sql
SHOW TABLES IN default;


database,tableName,isTemporary
default,bronze_admission_data,false
default,bronze_emergency_room_data,false
default,bronze_er_wait_time,false
default,bronze_events,false
default,events_delta,false
default,events_table,false
default,gold_product_revenue,false
default,kaggle,false
default,silver_er_forecast,false
default,silver_er_wait_time,false


In [0]:
%sql
DESCRIBE TABLE default.silver_er_wait_time;


col_name,data_type,comment
visit_id,string,null
patient_id,string,null
hospital_id,string,null
hospital_name,string,null
Region,string,null
visit_date,timestamp,null
day_of_week,string,null
season,string,null
time_of_day,string,null
urgency_level,string,null


In [0]:
%sql
SELECT COUNT(*) FROM default.silver_er_wait_time;


COUNT(*)
5000


In [0]:
%sql
SELECT COUNT(*) FROM default.silver_er_forecast;


COUNT(*)
300


In [0]:
%sql
SELECT
  COUNT(*) AS total_rows,
  SUM(CASE WHEN admission_date IS NULL THEN 1 ELSE 0 END) AS null_admission_dates
FROM default.silver_hospital_admissions;


total_rows,null_admission_dates
0,null


In [0]:
%sql
CREATE OR REPLACE TABLE default.gold_department_daily_load AS
SELECT
    CAST(w.visit_date AS DATE)           AS date,
    'Emergency'                           AS department,

    COUNT(DISTINCT w.patient_id)          AS total_admissions,

    -- operational metrics
    AVG(w.time_to_triage_min)             AS avg_triage_time_min,
    AVG(w.time_to_registration_min)       AS avg_registration_time_min,
    AVG(w.nurse_patient_ratio)            AS avg_nurse_patient_ratio,
    AVG(w.specialist_availability)        AS avg_specialist_availability,

    -- forecasted load
    COALESCE(SUM(f.patient_count),0)      AS forecasted_er_load,

    CURRENT_DATE()                        AS processing_date

FROM default.silver_er_wait_time w

LEFT JOIN default.silver_er_forecast f
    ON CAST(w.visit_date AS DATE) = CAST(f.date AS DATE)

GROUP BY CAST(w.visit_date AS DATE);

num_affected_rows,num_inserted_rows


In [0]:
%sql
SELECT * 
FROM default.gold_department_daily_load
ORDER BY date
LIMIT 20;


date,department,total_admissions,avg_triage_time_min,avg_registration_time_min,avg_nurse_patient_ratio,avg_specialist_availability,forecasted_er_load,processing_date
2024-01-01,Emergency,10,36.4,21.3,3.2,5.0,960,2026-01-30
2024-01-02,Emergency,16,23.9375,9.1875,2.75,4.75,1392,2026-01-30
2024-01-03,Emergency,13,30.615384615384617,10.615384615384615,2.923076923076923,2.6153846153846154,1079,2026-01-30
2024-01-04,Emergency,10,39.3,17.4,3.4,2.6,910,2026-01-30
2024-01-05,Emergency,12,34.916666666666664,17.333333333333332,3.6666666666666665,3.75,948,2026-01-30
2024-01-06,Emergency,14,14.714285714285714,8.714285714285714,3.2857142857142856,2.357142857142857,1134,2026-01-30
2024-01-07,Emergency,13,33.07692307692308,16.076923076923077,3.8461538461538463,4.153846153846154,884,2026-01-30
2024-01-08,Emergency,16,25.125,13.0625,2.4375,5.0625,1296,2026-01-30
2024-01-09,Emergency,18,33.333333333333336,17.22222222222222,3.2222222222222223,6.555555555555555,1224,2026-01-30
2024-01-10,Emergency,10,26.3,14.7,3.2,5.0,740,2026-01-30


In [0]:
%sql
SELECT COUNT(*) FROM default.gold_department_daily_load;


COUNT(*)
365


In [0]:
%sql
SELECT * 
FROM default.gold_department_load_risk
ORDER BY date;

date,department,total_admissions,avg_triage_time_min,avg_registration_time_min,avg_nurse_patient_ratio,avg_specialist_availability,forecasted_er_load,processing_date,overload_risk
2024-01-01,Emergency,10,36.4,21.3,3.2,5.0,960,2026-01-30,HIGH
2024-01-02,Emergency,16,23.9375,9.1875,2.75,4.75,1392,2026-01-30,MEDIUM
2024-01-03,Emergency,13,30.615384615384617,10.615384615384615,2.923076923076923,2.6153846153846154,1079,2026-01-30,MEDIUM
2024-01-04,Emergency,10,39.3,17.4,3.4,2.6,910,2026-01-30,HIGH
2024-01-05,Emergency,12,34.916666666666664,17.333333333333332,3.6666666666666665,3.75,948,2026-01-30,MEDIUM
2024-01-06,Emergency,14,14.714285714285714,8.714285714285714,3.2857142857142856,2.357142857142857,1134,2026-01-30,MEDIUM
2024-01-07,Emergency,13,33.07692307692308,16.076923076923077,3.8461538461538463,4.153846153846154,884,2026-01-30,MEDIUM
2024-01-08,Emergency,16,25.125,13.0625,2.4375,5.0625,1296,2026-01-30,MEDIUM
2024-01-09,Emergency,18,33.333333333333336,17.22222222222222,3.2222222222222223,6.555555555555555,1224,2026-01-30,HIGH
2024-01-10,Emergency,10,26.3,14.7,3.2,5.0,740,2026-01-30,MEDIUM
